### Using the Dickey-Fuller Method to Verify Seasonality, Stationanarity, and Constancy of Our Datasets Before the Model

In [56]:
from datetime import datetime
import pandas as pd
from statsmodels.tsa.stattools import adfuller
import pmdarima as pm
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import statsmodels.api as sm

In [2]:
# load the dataset
df = pd.read_csv('../datasets/scoring/final_aggregation.csv', parse_dates=['date'])

In [3]:
test_df = df[(df.date <= pd.to_datetime('2016-12-31')) & (df.date >= pd.to_datetime('2016-01-01'))]
train_df = df[(df.date <= pd.to_datetime('2019-01-01')) & (df.date > pd.to_datetime('2016-12-31'))]

In [4]:
test_df.fillna(0, inplace=True)
train_df.fillna(0, inplace=True)

/var/folders/5d/fdmfhfr55d5f2r62ltcl50vh0000gn/T/ipykernel_41825/1154098784.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.fillna(0, inplace=True)
/var/folders/5d/fdmfhfr55d5f2r62ltcl50vh0000gn/T/ipykernel_41825/1154098784.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.fillna(0, inplace=True)


In [5]:
test_df.set_index('date', inplace=True)
train_df.set_index('date', inplace=True)

In [6]:
test_df.to_csv('../datasets/training/SARIMA/test_df.csv', index=False)
train_df.to_csv('../datasets/training/SARIMA/train_df.csv', index=False)

In [71]:
def dickey_fuller_test(df, exempt_cols=['date'], sig_level=0.05):
    """
    Perform Dickey-Fuller test for stationarity on all columns in a DataFrame.

    Parameters
    ----------
    df : DataFrame
    Input time series data 
    exempt_cols : list, default ['date']
    Columns to skip testing
    sig_level : float, default 0.05 
    Significance level for testing

    Returns
    -------
    results : dict
    Dictionary containing test results for each column 
    """
    # Dictionary to store results
    results = {} 

    # Loop through columns
    for col in df.columns:

        # Skip exempt columns
        if col in exempt_cols:
            continue

        # Print progress  
        print(f"{datetime.now()}: Testing {col} for stationarity...")

        try:
            # Perform ADF test
            adf_result = adfuller(df[col])
            
        except:
            # Handle errors
            print(f"Error testing {col}. Skipping.")
            continue

        # Store results    
        results[col] = {
            'adf_stat': adf_result[0],
            'pvalue': adf_result[1],
            'sig_level': sig_level 
        }

        # Print conclusions
        pvalue = results[col]['pvalue']
        sig = results[col]['sig_level']
            
        stationary = pvalue < sig  
        print(f"{col} stationary: {stationary} (p-value: {pvalue:.3f})")
        
    return results

In [72]:
exempt_cols = ['date', 'hour', 'area', 'cta_stations', 'police_stations', 'bus_stations', 'unemployment', 
            'per_capita_income', 'no_hs_dip', 'gov_depend', 'crowded_housing', 'below_pov', 'bike_stations']

adf_results = dickey_fuller_test(test_df, exempt_cols)
adf_results

2023-10-24 17:35:05.796478: Testing non-violent for stationarity...
non-violent stationary: True (p-value: 0.000)
2023-10-24 17:42:06.290312: Testing violent for stationarity...
violent stationary: True (p-value: 0.000)
2023-10-24 17:49:00.310958: Testing train_rides for stationarity...
train_rides stationary: True (p-value: 0.000)
2023-10-24 17:55:53.402942: Testing bike_rides for stationarity...
bike_rides stationary: True (p-value: 0.000)
2023-10-24 18:02:50.953673: Testing lighting for stationarity...
lighting stationary: False (p-value: 0.158)
2023-10-24 18:09:39.281249: Testing vacant_buildings for stationarity...
vacant_buildings stationary: True (p-value: 0.000)


{'non-violent': {'adf_stat': -35.84644081359766,
  'pvalue': 0.0,
  'sig_level': 0.05},
 'violent': {'adf_stat': -39.03089699079734, 'pvalue': 0.0, 'sig_level': 0.05},
 'train_rides': {'adf_stat': -18.2608600940507,
  'pvalue': 2.3312991666018573e-30,
  'sig_level': 0.05},
 'bike_rides': {'adf_stat': -26.917965126201302,
  'pvalue': 0.0,
  'sig_level': 0.05},
 'lighting': {'adf_stat': -2.3449276970859816,
  'pvalue': 0.1578868690956337,
  'sig_level': 0.05},
 'vacant_buildings': {'adf_stat': -15.769658866629179,
  'pvalue': 1.165106212455214e-28,
  'sig_level': 0.05}}

In [ ]:
def plot_acf_pacf(df, exempt_cols):

    """Generate ACF and PACF plots for each column in a DataFrame"""
    
    # Loop through columns
    for col in df.columns:

        if col not in exempt_cols:
            # Create ACF plot
            plt.subplot(211) 
            plot_acf(df[col], lags=20)
            plt.title(f"ACF - {col}")

            # Create PACF plot        
            plt.subplot(212)
            plot_pacf(df[col], lags=20)
            plt.title(f"PACF - {col}")

            # Show plot
            plt.tight_layout()
            plt.show()

In [ ]:
plot_acf_pacf(test_df, exempt_cols)

In [69]:
def find_auto_arima(series, adf_result, sig_val=0.05, max_p=3, max_q=3, max_d=1, stepwise=True):
    """
    Search for optimal ARIMA parameters for a time series.

    Parameters
    ----------
    series : pd.Series
    The time series data
    max_p, max_q : int
    Maximum p and q values to check 
    max_d : int
    Maximum number of differences allowed 
    stepwise : bool
    Whether to use stepwise model selection 
    """

    max_d = 0 if adf_result < sig_val else 1

    print("Fitting models...")
    mods = pm.auto_arima(series, start_p=0, start_q=0, max_p=max_p, max_q=max_q,
                        max_d=max_d, stepwise=stepwise, suppress_warnings=True)

    print("Selecting best model...")
    best_mod = mods.apply(lambda x: x.aic()).idxmin()   

    print("Best model found: {}".format(best_mod.order))
    return best_mod.order

In [ ]:
find_auto_arima(test_df['violent'], adf_results['violent']['pvalue'], adf_results['violent']['sig_level'])

In [9]:
def ARIMA_model(df, col, p, d, q, P, D, Q, stationary_state, m=24):
    """
    Fit a SARIMA or ARIMA model based on the stationarity state.

    Parameters:
    - df: DataFrame containing the time series data.
    - col: Name of the column to model.
    - p, d, q: Parameters for the ARIMA component.
    - P, D, Q: Parameters for the seasonal component (SARIMA).
    - seasonal_state: True if data is stationary, False otherwise.
    - m: Seasonal period for SARIMA model.

    Returns:
    - results: Fitted SARIMA or ARIMA model.
    """
    if stationary_state:
        print(f'{datetime.now()}: Data for column "{col}" is stationary. Fitting SARIMA model.')
        model = SARIMAX(df[col],
                        order=(p, d, q),
                        seasonal_order=(P, D, Q, m))
    else:
        print(f'{datetime.now()}: Data for column "{col}" is not stationary. Fitting ARIMA model instead.')
        model = ARIMA(df[col], order=(p, d, q))

    results = model.fit()
    
    return results